# Web Scraping Intro

Sometimes, data viewable on the web may not be available through an API or downloadable as csv or otherwise. In these cases, scraping that data might be possible.

In this notebook, we'll see how we can retrieve the contents of a website and then parse the resulting HTML to extract the data we want.

For this, we'll again be using the [_requests_](https://requests.readthedocs.io/en/master/) library.

In [1]:
import requests

Let's say we want to pull some data from http://en.wikipedia.org/wiki/Turing_Award. 

I can start by sending a get request for the contents of the site. Note that Wikipedia requires us to include a [User-Agent header](https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Headers/User-Agent). For more information about HTTP headers see https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Headers.

In [2]:
URL = 'http://en.wikipedia.org/wiki/Turing_Award'

headers = {
    "User-Agent": "MyPythonScraper"
}

response = requests.get(URL, headers=headers)

We can check the status code using the `status_code` attribute.

In [3]:
response.status_code

200

A 200 status code is the standard response for a successful request.  

Let's see what this request returned.

In [4]:
response.text

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Turing Award - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-

It is very hard to decipher the above text. Luckily for us, the [_Beautiful Soup_](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library comes to the rescue. This library assists us in parsing HTML into something usable.

In [5]:
from bs4 import BeautifulSoup

First, we can soupify our response text. Since we are working with HTML, we can specify that we need the html parser.

In [6]:
soup = BeautifulSoup(response.text)

Now, we can print it out in a slightly more readable form.

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Turing Award - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-en

What we are looking at is the HTML for this page. This is rendered by your browser into the Wikipedia page that you see.

<img src="assets/html.png">


If you navigate to this page in your browser, you can view page source or inspect elements to see the underlying HTML.

Beautiful Soup lets us search through this HTML and extract out the contents we want by tag.  

Say we wanted to find the title of this page. We can accomplish this by using the `.find` method on our soup, telling it that we want to find the first `title` tag.

In [8]:
soup.find('title')

<title>Turing Award - Wikipedia</title>

Notice that this returns a bs4 Tag object.

In [9]:
type(soup.find('title'))

bs4.element.Tag

To extract out the text, you can use the `.text` attribute.

In [10]:
soup.find('title').text

'Turing Award - Wikipedia'

The `.find` method find the first matching tag. 

We can find _all_ elements with a particular tag using the `.findAll(<tag>)` method. Say we want to find all images. We'll look for the `img` tag.

In [11]:
images = soup.findAll('img')
images

[<img alt="" aria-hidden="true" class="mw-logo-icon" height="50" src="/static/images/icons/wikipedia.png" width="50"/>,
 <img alt="Wikipedia" class="mw-logo-wordmark" src="/static/images/mobile/copyright/wikipedia-wordmark-en.svg" style="width: 7.5em; height: 1.125em;"/>,
 <img alt="The Free Encyclopedia" class="mw-logo-tagline" height="13" src="/static/images/mobile/copyright/wikipedia-tagline-en.svg" style="width: 7.3125em; height: 0.8125em;" width="117"/>,
 <img alt="This is a featured list. Click here for more information." class="mw-file-element" data-file-height="443" data-file-width="466" decoding="async" height="19" src="//upload.wikimedia.org/wikipedia/en/thumb/e/e7/Cscr-featured.svg/20px-Cscr-featured.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/e/e7/Cscr-featured.svg/40px-Cscr-featured.svg.png 1.5x" width="20"/>,
 <img alt="Statue of Alan Turing" class="mw-file-element" data-file-height="3000" data-file-width="1993" decoding="async" height="376" src="//upload.w

Let's look closer at the first image.

In [12]:
first_image = images[0]
first_image

<img alt="" aria-hidden="true" class="mw-logo-icon" height="50" src="/static/images/icons/wikipedia.png" width="50"/>

You can access attributes of a Tag object in the same way that you would access values from a dictionary.

In [13]:
first_image['src']

'/static/images/icons/wikipedia.png'

You can also safely access attributes using `.get`. This might be useful if, for example, you aren't sure if a particular Tag or all tags had a certain attribute.

In [14]:
# Non-safe
first_image['class']

['mw-logo-icon']

In [15]:
# Safe
first_image.get('class')

['mw-logo-icon']

You can also specify a default value when using `get`.

In [16]:
first_image.get('class', default = 'No Class')

['mw-logo-icon']

If you want to grab a particular attribute for all images, an easy way to do so is with a list comprehension.

In [17]:
image_srcs = [x.get('src') for x in images]

In [18]:
image_srcs

['/static/images/icons/wikipedia.png',
 '/static/images/mobile/copyright/wikipedia-wordmark-en.svg',
 '/static/images/mobile/copyright/wikipedia-tagline-en.svg',
 '//upload.wikimedia.org/wikipedia/en/thumb/e/e7/Cscr-featured.svg/20px-Cscr-featured.svg.png',
 '//upload.wikimedia.org/wikipedia/commons/thumb/7/76/Turing_statue_Surrey.jpg/250px-Turing_statue_Surrey.jpg',
 '//upload.wikimedia.org/wikipedia/en/e/e3/Turing_Award%2C_on_display_at_Nokia_Bell_Labs%2C_NJ%2C_USA%2C_Aug_2025.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/1/1d/No_image.svg/80px-No_image.svg.png',
 '//upload.wikimedia.org/wikipedia/commons/thumb/3/36/Maurice_Vincent_Wilkes_1980_%283%2C_cropped%29.jpg/120px-Maurice_Vincent_Wilkes_1980_%283%2C_cropped%29.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/1/1d/No_image.svg/80px-No_image.svg.png',
 '//upload.wikimedia.org/wikipedia/commons/thumb/b/bd/Marvin_Minsky_at_OLPCc.jpg/120px-Marvin_Minsky_at_OLPCc.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb

We can further navigate the html tree to extract out other bits of information.

When scraping from a web page, you should make use of "View Page Source" and/or "Inspect Element" in your web browswer.

For example, let's say we want to look at the second header on the page.

In [19]:
soup.findAll('header')

[<header class="vector-header mw-header no-font-mode-scale">
 <div class="vector-header-start">
 <nav aria-label="Site" class="vector-main-menu-landmark">
 <div class="vector-dropdown vector-main-menu-dropdown vector-button-flush-left vector-button-flush-right" id="vector-main-menu-dropdown" title="Main menu">
 <input aria-haspopup="true" aria-label="Main menu" class="vector-dropdown-checkbox" data-event-name="ui.dropdown-vector-main-menu-dropdown" id="vector-main-menu-dropdown-checkbox" role="button" type="checkbox"/>
 <label aria-hidden="true" class="vector-dropdown-label cdx-button cdx-button--fake-button cdx-button--fake-button--enabled cdx-button--weight-quiet cdx-button--icon-only" for="vector-main-menu-dropdown-checkbox" id="vector-main-menu-dropdown-label"><span class="vector-icon mw-ui-icon-menu mw-ui-icon-wikimedia-menu"></span>
 <span class="vector-dropdown-label-text">Main menu</span>
 </label>
 <div class="vector-dropdown-content">
 <div class="vector-unpinned-container" i

In [20]:
soup.findAll('header')[1]

<header class="mw-body-header vector-page-titlebar no-font-mode-scale">
<nav aria-label="Contents" class="vector-toc-landmark">
<div class="vector-dropdown vector-page-titlebar-toc vector-button-flush-left" id="vector-page-titlebar-toc" title="Table of Contents">
<input aria-haspopup="true" aria-label="Toggle the table of contents" class="vector-dropdown-checkbox" data-event-name="ui.dropdown-vector-page-titlebar-toc" id="vector-page-titlebar-toc-checkbox" role="button" type="checkbox"/>
<label aria-hidden="true" class="vector-dropdown-label cdx-button cdx-button--fake-button cdx-button--fake-button--enabled cdx-button--weight-quiet cdx-button--icon-only" for="vector-page-titlebar-toc-checkbox" id="vector-page-titlebar-toc-label"><span class="vector-icon mw-ui-icon-listBullet mw-ui-icon-wikimedia-listBullet"></span>
<span class="vector-dropdown-label-text">Toggle the table of contents</span>
</label>
<div class="vector-dropdown-content">
<div class="vector-unpinned-container" id="vecto

Similar to using `find` and `findall` in the full soup, we can use the `.find` method just within a Tag.

In [21]:
soup.findAll('header')[1].find('h1')

<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-main">Turing Award</span></h1>

In [22]:
soup.findAll('header')[1].find('h1').get('id')

'firstHeading'

In [23]:
soup.findAll('header')[1].find('h1').text

'Turing Award'

In [24]:
soup.findAll('header')[1].findAll('div')

[<div class="vector-dropdown vector-page-titlebar-toc vector-button-flush-left" id="vector-page-titlebar-toc" title="Table of Contents">
 <input aria-haspopup="true" aria-label="Toggle the table of contents" class="vector-dropdown-checkbox" data-event-name="ui.dropdown-vector-page-titlebar-toc" id="vector-page-titlebar-toc-checkbox" role="button" type="checkbox"/>
 <label aria-hidden="true" class="vector-dropdown-label cdx-button cdx-button--fake-button cdx-button--fake-button--enabled cdx-button--weight-quiet cdx-button--icon-only" for="vector-page-titlebar-toc-checkbox" id="vector-page-titlebar-toc-label"><span class="vector-icon mw-ui-icon-listBullet mw-ui-icon-wikimedia-listBullet"></span>
 <span class="vector-dropdown-label-text">Toggle the table of contents</span>
 </label>
 <div class="vector-dropdown-content">
 <div class="vector-unpinned-container" id="vector-page-titlebar-toc-unpinned-container">
 </div>
 </div>
 </div>,
 <div class="vector-dropdown-content">
 <div class="vec

Now, let's look for the table containing the Turing Award winners.

Using `.findAll` reveals that there are multiple tables on the page.

In [25]:
len(soup.findAll('table'))

7

If we know a bit more about what we are looking for, we can include an `attrs` argument and pass a dictionary. 

Go to the Turing award page in your browser, right click on the top of the table and choose "Inspect". You will notice that this table is defined with tag `<table class="wikitable">.` Armed with this information, we can narrow down our search.

In [26]:
soup.find('table', attrs={'class' : 'wikitable'})

<table class="wikitable sortable">
<caption>Recipients of the ACM Turing award
</caption>
<tbody><tr>
<th scope="col">Year
</th>
<th scope="col">Recipient(s)
</th>
<th class="unsortable" scope="col">Photo
</th>
<th class="unsortable" scope="col">Rationale
</th>
<th scope="col">Affiliated institute(s)
</th></tr>
<tr>
<th scope="row">1966
</th>
<td><span data-sort-value="Perlis, Alan"><span class="vcard"><span class="fn"><a href="/wiki/Alan_Perlis" title="Alan Perlis">Alan Perlis</a></span></span></span>
</td>
<td align="center"><span typeof="mw:File"><a class="mw-file-description" href="/wiki/File:No_image.svg"><img class="mw-file-element" data-file-height="1" data-file-width="1" decoding="async" height="80" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1d/No_image.svg/80px-No_image.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1d/No_image.svg/120px-No_image.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/1d/No_image.svg/160px-No_image.svg.png 

We can display the table by importing the `HTML` function.

In [27]:
table_html = str(soup.find('table', attrs={'class' : 'wikitable'}))

from IPython.core.display import HTML

HTML(table_html)

Year,Recipient(s),Photo,Rationale,Affiliated institute(s)
1966,Alan Perlis,,"""For his influence in the area of advanced computer programming techniques and compiler construction""[16][17]",Carnegie Mellon University
1967,Maurice Wilkes,,"For contributions including being ""the builder and designer of the EDSAC, the second computer with an internally stored program"" and introducing program libraries (together with David Wheeler and Stanley Gill)[18][19]",University of Cambridge
1968,Richard Hamming,,"""For his work on numerical methods, automatic coding systems, and error-detecting and error-correcting codes""[20][21]",Bell Labs
1969,Marvin Minsky,,"""For his central role in creating, shaping, promoting, and advancing the field of artificial intelligence""[22][23]",Massachusetts Institute of Technology
1970,James H. Wilkinson,,"""For his research in numerical analysis to facilitate the use of the high-speed digital computer, having received special recognition for his work in computations in linear algebra and 'backward' error analysis""[24][25]",National Physical Laboratory
1971,John McCarthy,,"Award citation refers to McCarthy's lecture ""The Present State of Research on Artificial Intelligence""[26][27]",Stanford University
1972,Edsger W. Dijkstra,,"""For fundamental contributions to programming as a high, intellectual challenge; for eloquent insistence and practical demonstration that programs should be composed correctly, not just debugged into correctness; for illuminating perception of problems at the foundations of program design""[28][29]",Centrum Wiskunde & InformaticaEindhoven University of TechnologyUniversity of Texas at Austin
1973,Charles Bachman,,"""For his outstanding contributions to database technology""[30][31]","General Electric Research Laboratory (now under Groupe Bull, an Atos company)"
1974,Donald Knuth,,"""For his major contributions to the analysis of algorithms and the design of programming languages, and in particular for his contributions to 'The Art of Computer Programming' through his well-known books in a continuous series by this title""[32][33]","California Institute of TechnologyCenter for Communications Research, Center for Communications and Computing, Institute for Defense AnalysesStanford University"
1975,Allen Newell,,"In collaboration with J. C. Shaw and others, for ""basic contributions to artificial intelligence, the psychology of human cognition, and list processing.""[34][35][36]",RAND CorporationCarnegie Mellon University


However, this does not give us a way to work with the data in the table, only to display it.

If we want to interact with the table, we can use the _pandas_ `read_html` method.

In [28]:
import pandas as pd
from io import StringIO

In [29]:
pd.read_html(StringIO(str(soup.find('table', attrs={'class' : 'wikitable'}))))[0]

,Year,Recipient(s),Photo,Rationale,Affiliated institute(s)
0,1966,Alan Perlis,NaN,"""For his influence in the area of advanced com...",Carnegie Mellon University
1,1967,Maurice Wilkes,NaN,"For contributions including being ""the builder...",University of Cambridge
2,1968,Richard Hamming,NaN,"""For his work on numerical methods, automatic ...",Bell Labs
3,1969,Marvin Minsky,NaN,"""For his central role in creating, shaping, pr...",Massachusetts Institute of Technology
4,1970,James H. Wilkinson,NaN,"""For his research in numerical analysis to fac...",National Physical Laboratory
...,...,...,...,...,...
74,2021,Jack Dongarra,NaN,"""For pioneering contributions to numerical alg...",Argonne National Laboratory Oak Ridge National...
75,2022,Robert Metcalfe,NaN,"""For the invention, standardization, and comme...","Massachusetts Institute of Technology, Harvard..."
76,2023,Avi Wigderson,NaN,"""For foundational contributions to the theory ...","Institute for Advanced Study, Princeton Univer..."
77,2024,Andrew Barto,NaN,"""For developing the conceptual and algorithmic...",University of Massachusetts Amherst
